<a href="https://colab.research.google.com/github/shizoda/education/blob/main/machine_learning/text/Using_LLM_via_OpenRouter_and_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 はじめに: LangChain からの LLM 呼び出し

このノートブックは、**LangChain (ラングチェーン)** を使って、**OpenRouter (オープンルーター)** 経由でさまざまな LLM (大規模言語モデル) の API を呼び出す練習をするための教材です。

## 🤔 LangChain とは？

**LangChain (ラングチェーン)** は、LLM を使ったアプリケーション開発を簡単にするためのフレームワークです。

* **コンポーネント:** LLM を呼び出す機能、プロンプト（指示文）を作る機能、LLM の回答を整理する機能など、よく使う部品（コンポーネント）が揃っています。
* **チェーン:** これらのコンポーネントを「チェーン（鎖）」のようにつなげることで、「PDF を読み込んで質問に答える」や「データベースと連携して回答する」といった複雑な処理を、比較的少ないコードで実現できます。

---
## 🌎 1. OpenRouter とは？

OpenRouter は、**「LLM のルーター（仲介・振り分け役）」** のようなサービスです。

通常、OpenAI のモデル (GPT) を使いたい場合は OpenAI の API キーが、Anthropic のモデル (Opus, Sonnet など) を使いたい場合は Anthropic の API キーが... というように、モデル提供会社ごとに契約とAPIキーが必要で、呼び出し方も少しずつ違います。

OpenRouter を使うと、以下の利点があります。

1.  **単一の API キー:** OpenRouter の API キーが 1 つあれば、OpenRouter が提携している数十種類以上のモデル（OpenAI, Google, Anthropic, Meta, Mistral, Qwen, DeepSeek など）をすべて呼び出せます。
2.  **共通の API 形式:** LangChain と組み合わせることで、どのモデルも同じような（統一された）方法で呼び出せます。
3.  **コスト管理:** 利用料金は OpenRouter にまとめて支払う（クレジットカード登録）だけで済み、モデルごとのコストもダッシュボードで一目でわかります。
4.  **安価・オープンなモデルへのアクセス:** OpenAI の GPT，Google の Gemini のような著名なモデルだけでなく、`Llama ` や `Qwen`、`DeepSeek` といった、非常に安価（または無料枠がある）あるいは高性能なモデルにも簡単にアクセスできます。オープンソースのものもあります。

この演習では、この OpenRouter を LangChain と組み合わせて利用する方法を学びます。

---
## 🔑 2. 準備: API キーとライブラリのインストール

API を呼び出すための準備をします。

### ステップ 2-1: OpenRouter の API キーを Colab に設定する

（内容は変更ありません。`OPENROUTER_API_KEY` をシークレットに設定してください。）

1.  [OpenRouter のサイト](https://openrouter.ai/keys) にアクセスし、アカウント登録・ログインします。
2.  （初回利用時）クレジットカード情報を登録し、利用上限額（例: $10）を設定します。
3.  「Keys」ページで「+ Create Key」を押し、キーを発行します。（`sk-or-...` で始まる文字列です）
4.  Colab の画面左側にある「🔑」（鍵マーク）アイコンをクリックします。
5.  「新しいシークレットを追加」を押します。
    * **名前 (name):** `OPENROUTER_API_KEY`
    * **値 (value):** 先ほどコピーした `sk-or-...` のキー
6.  「ノートブックへのアクセス」を **オン (有効)** にします。

### ステップ 2-2: 必要なライブラリのインストール

LangChain のコア機能である `langchain` と、OpenAI 互換 API (OpenRouter を含む) に接続するための `langchain-openai` をインストールします。

In [ ]:
# 練習 2-2: 必要なライブラリをインストールします
# (ヒント: !pip install ...)

!pip install langchain langchain-openai

print("langchain と langchain-openai のインストールが完了しました。")

---
## 🔌 3. LangChain クライアントの初期化

API を呼び出すための「クライアント」を LangChain を使って設定します。

OpenRouter は OpenAI と互換性のある API を提供しているため、LangChain では `langchain_openai` パッケージの `OpenAI`（Text Completion用）と `ChatOpenAI`（Chat Completion用）クラスをそのまま利用できます。

**重要なポイント:**
* `base_url`: 接続先を OpenAI 公式ではなく、OpenRouter の URL (`https://openrouter.ai/api/v1`) に指定します。
* `api_key`: Colab シークレットから読み込んだ OpenRouter の API キーを指定します。

### 📑 サンプルコード (クライアント初期化)

```python
from langchain_openai import OpenAI, ChatOpenAI
from google.colab import userdata

# 1. Colab シークレットからキーを取得
my_openrouter_key = userdata.get("OPENROUTER_API_KEY")

# 2. OpenRouter の API エンドポイント URL
openrouter_base_url = "[https://openrouter.ai/api/v1](https://openrouter.ai/api/v1)"

# 3. Text Completion 用クライアントを初期化
# (この時点ではモデルを指定する必要はありません)
llm_client = OpenAI(
  base_url=openrouter_base_url,
  api_key=my_openrouter_key
)

# 4. Chat Completion 用クライアントを初期化
chat_client = ChatOpenAI(
  base_url=openrouter_base_url,
  api_key=my_openrouter_key
)

print("LangChain クライアント (OpenAI, ChatOpenAI for OpenRouter) の初期化準備ができました。")

In [ ]:
from langchain_openai import OpenAI, ChatOpenAI
from google.colab import userdata

# 練習 3-1: Colab シークレットから "OPENROUTER_API_KEY" を取得
my_openrouter_key = userdata.get("OPENROUTER_API_KEY") # 👈 ここにシークレット名

# 練習 3-2: OpenRouter の URL を変数に格納
openrouter_base_url = "https://openrouter.ai/api/v1" # 👈 OpenRouter の URL

# 練習 3-3: Text Completion 用クライアント (llm_client) を初期化
llm_client = OpenAI(
    base_url= openrouter_base_url , # 👈 ここに openrouter_base_url 変数を入力
    api_key = my_openrouter_key  # 👈 ここに my_openrouter_key 変数を入力
)

# 練習 3-4: Chat Completion 用クライアント (chat_client) を初期化
chat_client = ChatOpenAI(
    base_url= openrouter_base_url , # 👈 ここに openrouter_base_url 変数を入力
    api_key= my_openrouter_key  # 👈 ここに my_openrouter_key 変数を入力
)

print("LangChain クライアント (OpenAI, ChatOpenAI for OpenRouter) が初期化されました。")

## 📜 4. モデルタイプ1: Text Completion (単純なテキスト補完)

これは、LLM の API としては古い形式ですが、「与えられた文章の**続き**をそのまま書く」というモデルの基本的な動作を理解するのに役立ちます。

* **LangChain API:** `llm_client.invoke()` メソッドを使います。
* **注意点:** `langchain_openai` の `OpenAI` クラスは、呼び出し時 (`invoke`) にモデルを変更する機能がありません。そのため、**使用するモデルを指定してクライアントを都度作成する**必要があります。
* **モデル例:** `nousresearch/nous-hermes-llama-2-7b` (Free Tier), `deepseek/deepseek-chat-v3-0324`

### 📑 サンプルコード (Text Completion)

```python
# (base_url と api_key は練習3のものを使います)

# 1. Text Completion をサポートするモデル名を指定
model_name = "deepseek/deepseek-chat-v3-0324"

# 2. 続きを書いてほしいプロンプト
prompt_text = "昔々あるところに、"

# 3. API を呼び出す (モデルを指定してクライアントを都度作成)
# (注: 教材の構成上、都度作成していますが、通常は練習3の llm_client をモデル固定で使います)
temp_llm_client = OpenAI(
    base_url=openrouter_base_url,
    api_key=my_openrouter_key,
    model_name=model_name, # 👈 使用するモデルをここで指定
    max_tokens=50
)
response_text = temp_llm_client.invoke(prompt_text)


# 4. 結果（続きの文章）を表示
# .invoke() は補完された文字列 (str) を直接返します
print(f"--- {model_name} が生成した続き ---")
print(prompt_text + response_text)

In [ ]:
# (chat_client, openrouter_base_url, my_openrouter_key は練習3で定義したものを使います)
# (ChatOpenAI を使うため、HumanMessage をインポートします)
from langchain_core.messages import HumanMessage

# 練習 4-1: モデル名を指定 (Chat API に対応したモデル)
model_name = "deepseek/deepseek-chat-v3-0324" # 👈 Chat API 対応モデル

# 練習 4-2: 続きを書いてほしいプロンプト
prompt_text = "昔々あるところに、" # 👈 プロンプト

# 練習 4-3: ChatOpenAI クライアント (chat_client) を使って invoke() を呼び出す
# (ヒント: 練習5 と同様の方法を使います)

# (1) プロンプトを HumanMessage に変換します
messages = [
    HumanMessage(content=prompt_text)
]

# (2) chat_client.invoke を呼び出します
response_message = chat_client.invoke(
    messages,
    model=model_name,
    max_tokens=30
)

# 練習 4-4: 結果（プロンプト + 続き）を表示
# (ヒント: 回答は response_message.content に入っています)
generated_text = response_message.content # 👈 .content からテキストを取得

print(f"--- {model_name} による補完結果 ---")
# Chat API の場合、モデルはプロンプトを繰り返さないことが多いですが、
# 続きものであることを示すために prompt_text を足して表示します。
print(prompt_text + generated_text)

---
## 🧠 5. モデルタイプ2: Chat Completion (対話・指示実行)

現在、API 利用の主流となっている形式です。
単純な続きを書くだけでなく、役割を与えたり、複雑な指示 (Reasoning) に従わせたりすることができます。

* **LangChain API:** `chat_client.invoke()` メソッドを使います。
* **特徴:** `messages` リストを使って、役割ごとに発言を渡します。LangChain では、`Message` オブジェクトを使うのが標準です。
    * `SystemMessage`: AI の役割や前提条件を指示します (例: 「あなたはPythonの専門家です」)。
    * `HumanMessage`: ユーザーからの質問や指示です。
    * `AIMessage`: (オプション) AI の過去の応答例です。
* **利便性:** `ChatOpenAI` クラスは、`invoke` メソッドの引数で `model=...` を指定することで、呼び出しごとにモデルを切り替えることができます。

### 📑 サンプルコード (Chat Completion)

```python
from langchain_core.messages import SystemMessage, HumanMessage

# (chat_client は練習3で初期化したものを使います)

# 1. Chat Completion モデル名を指定
model_name = "deepseek/deepseek-r1-0528-qwen3-8b"

# 2. 送信するメッセージのリストを作成 (LangChain 形式)
messages = [
    SystemMessage(content="あなたはPythonコーディングの専門家です。"),
    HumanMessage(content="Pythonで、リストの偶数だけを返す関数を書いてください。")
]

# 3. API を呼び出す (invoke メソッド)
response_message = chat_client.invoke(
    messages, # 👈 Message オブジェクトのリストを渡す
    model=model_name # 👈 呼び出し時にモデルを指定 (model_name ではなく model)
)

# 4. 結果（回答）を表示
# .invoke() は AIMessage オブジェクトを返します
# 回答内容は .content 属性に入っています
print(f"--- {model_name} からの回答 ---")
print(response_message.content)

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

# (chat_client は練習3で初期化したものを使います)

# 練習 5-1: モデル名を指定 (例: "deepseek/deepseek-r1-0528-qwen3-8b")
model_name = "deepseek/deepseek-r1-0528-qwen3-8b"# 👈 ここにチャットモデル名を入力

# 練習 5-2: メッセージリストを作成 (SystemMessage と HumanMessage を使う)
messages = [
    SystemMessage(content="あなたは日本語で回答する、親切なAIアシスタントです。"), # 👈 AIの役割
    HumanMessage(content="いまの日本の総理大臣は誰？")  # 👈 ユーザーの質問
]

# 練習 5-3: chat_client.invoke() を呼び出す
# (ヒント: 第1引数に messages, 引数に model=model_name を指定)
response_message = chat_client.invoke(
    messages , # 👈 メッセージリスト変数
    model= model_name # 👈 モデル名変数 (引数名は model)
)

# 練習 5-4: 結果（回答）を表示
# (ヒント: 回答は response_message の .content 属性に入っています)
answer = response_message.content # 👈 ここに回答を取り出すコードを入力

print(f"--- {model_name} からの回答 ---")
print(answer)

---
## 🎉 6. まとめ

お疲れ様でした！この演習では、LangChain の `langchain-openai` ライブラリを使って OpenRouter 経由で2つの異なるタイプの LLM を呼び出す方法を学びました。

1.  **Text Completion (<code>OpenAI(...).invoke</code>):**
    * `OpenAI` クラスを使用。
    * `base_url` と `api_key` を指定して初期化。
    * `model_name` を指定して**都度クライアントを作成**し、`invoke` にプロンプト文字列 (str) を渡す。
    * 出力: 補完された文字列 (str)
2.  **Chat Completion (<code>chat_client.invoke</code>):**
    * `ChatOpenAI` クラスを使用。
    * `base_url` と `api_key` を指定してクライアントを初期化。
    * `invoke` に `Message` オブジェクトのリストと、**`model=...`** を渡す。
    * 出力: `AIMessage` オブジェクト (回答は `.content` 属性)

`langchain-openai` を利用することで、接続先 (`base_url`) を変えるだけで、OpenRouter が提供する多様なモデルにアクセスできることを確認できました。